## Python для работы с базами данных

Изменения в Vagrantfile:
1. Перед запуском виртуалки раскомментировать строку 50
```bash
config.vm.network "forwarded_port", guest: 5432, host: 5432, host_ip: "127.0.0.1"
```

2. На виртуалке меняем в конфигурационном файле postgresql метод аутентификации (в редакторе nano после внесения изменений нажимаем Ctrl+x и на вопрос отвечаем y + enter):
```bash
sudo nano /etc/postgresql/10/main/pg_hba.conf
```

Находим строчку (в конце файла)
```bash
# Database administrative login by Unix domain socket
local   all             postgres                                peer
```

меняем peer на md5
```bash
local   all             postgres                                md5
```

и разрешаем подключения извне (меняем 127.0.0.1/32 на 0.0.0.0/0)
```bash
# IPv4 local connections:
host    all             all             0.0.0.0/0            md5
```

В файле конфигураций разрешаем слушать все IP-адреса
```bash
sudo nano /etc/postgresql/10/main/postgresql.conf
# находим строчку с listen_addresses и меняем значение на
listen_addresses = '*'
```

В конце перезапускаем postgresql
```bash
sudo service postgresql restart
```

3. Заходим под пользователем postgres и добавляем в базу пользователя с правами на чтение
```bash
sudo su -l postgres
psql -U postgres -c "CREATE ROLE readaccess;"
psql -U postgres -c "GRANT SELECT ON ALL TABLES IN SCHEMA public TO readaccess;"
psql -U postgres -c "ALTER DEFAULT PRIVILEGES IN SCHEMA public GRANT SELECT ON TABLES TO readaccess;"
psql -U postgres -c "CREATE USER ds WITH PASSWORD 'ds';"
psql -U postgres -c "GRANT readaccess TO ds;"
```

In [ ]:
import psycopg2

In [ ]:
# инициализируем соединение с базой данных
conn = psycopg2.connect("dbname='postgres' user='ds' host='127.0.0.1' password='ds'")

In [ ]:
cur = conn.cursor()

In [ ]:
# для больших запросов
cur.itersize = 10000

In [ ]:
# отправляем запрос
cur.execute('SELECT COUNT(*) FROM ratings;')

In [ ]:
# получаем все строки ответа
cur.fetchall()

In [ ]:
cur.execute('SELECT * FROM ratings order by userid limit 10;')

In [ ]:
for row in cur:
    print(row)

### Немного про запросы в MySQL

In [ ]:
# для тяжелых запросов к MySQL можно использовать SSDictCursor
import pymysql

connection = pymysql.connect(
    host = 'dbhost',
    port = 3306,
    user = 'dbuser',
    password = 'dbpass',
    db = 'db',
    cursorclass = pymysql.cursors.SSDictCursor
)

with connection.cursor() as cursor:
    cursor.execute(sqlQuery)

    for line in cursor:
        print(line)

### Базы данных и pandas

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_sql('SELECT * FROM ratings limit 10;', conn)
data

## Персональные данные

In [ ]:
# плохой вариант для аналитики
database_rows = [
    {'name': 'Наталья', 'email': 'nata123@mail.ru', 'ip': '127.0.0.1'},
    {'name': 'Михаил', 'email': 'mikha@yandex.ru', 'ip': '127.0.0.1'},
]

In [ ]:
import hashlib

In [ ]:
hashlib.algorithms_available

In [ ]:
email = 'username@yandex.ru'

In [ ]:
m = hashlib.sha256()

m.update(bytes(email, 'utf-8'))
m.hexdigest()

### Передача сообщений
Проблема [Man in the middle](https://ru.wikipedia.org/wiki/%D0%90%D1%82%D0%B0%D0%BA%D0%B0_%D0%BF%D0%BE%D1%81%D1%80%D0%B5%D0%B4%D0%BD%D0%B8%D0%BA%D0%B0)

In [ ]:
message = 'Согласуете отпуск на следующей неделе?'

In [ ]:
correct_answer = 'Да'
corrupted_answer = 'Нет'

Заранее договоримся с руководителем добавлять в конец каждого сообщения. Кроме вас и руководителя никто не должен знать это значение.

In [ ]:
seed = '123'

К каждому сообщению добавляем seed и считаем хэш:

In [ ]:
sending_message = correct_answer + seed
sending_message

Вместе с sending_message передаем хэш

In [ ]:
m = hashlib.sha256()

m.update(bytes(sending_message, 'utf-8'))
m.hexdigest()

Если злоумышленник перехватит и изменит сообщение, то без seed он не сможет посчитать нужное значение хэша. Когда вы получаете сообщение, то добавляете к нему seed, считаете хэш и сравниваете с тем, что приложен к сообщению:

In [ ]:
received_message = 'Нет'

In [ ]:
m = hashlib.sha256()

m.update(bytes(received_message, 'utf-8'))
m.hexdigest()

Хэш получился другой, значит сообщение изменено!

В случае неизмененного сообщения хэши совпадут:

In [ ]:
m = hashlib.sha256()

m.update(bytes(correct_answer + seed, 'utf-8'))
m.hexdigest()

### Упражнение
Зашифруйте персональные данные в списке database_rows